In [108]:
import pandas as pd
import numpy as np

In [109]:
df = pd.read_csv('data_DGA.csv', index_col = 0)
df = df.iloc[:-1,:]
# df

df['h2'] = pd.to_numeric(df['h2'],errors='coerce')
df['ch4'] = pd.to_numeric(df['ch4'],errors='coerce')
df['c2h6'] = pd.to_numeric(df['c2h6'],errors='coerce')
df['c2h4'] = pd.to_numeric(df['c2h4'],errors='coerce')
df['act'] = pd.to_numeric(df['act'],errors='coerce')

#df.dtypes

In [110]:
df = df.reset_index(drop=True) #reset index

In [111]:
print(df['act'].nunique(dropna=True)) # 显示类别数量

6


In [112]:
X = df.iloc[:, :5]
y = df.iloc[:, -1]


In [113]:
# 划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=42)

In [114]:
# 使用 KNN 进行分类
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [115]:
# 评估 KNN 模型
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[ 5  1  0  0  0  0]
 [ 0  4  3  0  0  0]
 [ 0  1 13  0  0  0]
 [ 0  1  0 11  0  0]
 [ 0  0  0  0  9  2]
 [ 0  0  0  0  1  6]]
Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      0.83      0.91         6
         2.0       0.57      0.57      0.57         7
         3.0       0.81      0.93      0.87        14
         4.0       1.00      0.92      0.96        12
         5.0       0.90      0.82      0.86        11
         6.0       0.75      0.86      0.80         7

    accuracy                           0.84        57
   macro avg       0.84      0.82      0.83        57
weighted avg       0.85      0.84      0.84        57

Accuracy: 0.8421052631578947


In [116]:
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(
 learning_rate =0.05,
 n_estimators=700,
 max_depth=6,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 nthread=8,
 scale_pos_weight=2,
 seed=27)


In [117]:
y_train = y_train - 1
y_test = y_test - 1

In [118]:
xgb1.fit(X_train,y_train)

C:\Users\11094\AppData\Roaming\Python\Python310\site-packages\xgboost\core.py:158: UserWarning: [01:31:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=700, n_jobs=None, nthread=8,
              num_parallel_tree=None, ...)

In [119]:
preds = xgb1.predict(X_test) # 输出类别

In [121]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(preds,y_test) # 计算准确率
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.23%


In [122]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, preds)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, preds)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,preds)
print("Accuracy:",result2)

Confusion Matrix:
[[ 6  0  0  0  0  0]
 [ 0  4  3  0  0  0]
 [ 0  0 14  0  0  0]
 [ 0  0  0 12  0  0]
 [ 0  0  0  2  9  0]
 [ 0  0  0  0  0  7]]
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       1.00      0.57      0.73         7
         2.0       0.82      1.00      0.90        14
         3.0       0.86      1.00      0.92        12
         4.0       1.00      0.82      0.90        11
         5.0       1.00      1.00      1.00         7

    accuracy                           0.91        57
   macro avg       0.95      0.90      0.91        57
weighted avg       0.93      0.91      0.91        57

Accuracy: 0.9122807017543859


通常情况下，训练数据和测试数据的标准化应该使用相同的均值和标准差。因此，标准化测试数据时应使用 scaler.transform(X_test) 而不是 scaler.fit_transform(X_test)。这样可以确保训练数据和测试数据使用相同的标准化参数。

In [126]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ts = scaler.fit_transform(X_train)
X_tes = scaler.transform(X_test)
classifier.fit(X_ts,y_train)

KNeighborsClassifier(n_neighbors=1)

In [127]:
y_tes = classifier.predict(X_tes)

In [128]:
accuracy = accuracy_score(y_tes,y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.21%


In [130]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn_2 = KNeighborsClassifier()

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

clf = GridSearchCV(knn_2, hyperparameters, cv=10)

clf.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29],
                         'p': [1, 2]})

In [131]:
y_t = clf.predict(X_test)
accuracy = accuracy_score(y_t,y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.21%


In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=13, random_state=42)

clf.fit(X_train, y_train)

y_t = clf.predict(X_test)

In [133]:
accuracy = accuracy_score(y_t,y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 87.72%


In [134]:
for i in range(1,20):
    clf = RandomForestClassifier(max_depth=i, random_state=42)
    clf.fit(X_train, y_train)
    y_t = clf.predict(X_test)
    accuracy = accuracy_score(y_t,y_test)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    

Accuracy: 50.88%
Accuracy: 57.89%
Accuracy: 61.40%
Accuracy: 68.42%
Accuracy: 71.93%
Accuracy: 84.21%
Accuracy: 85.96%
Accuracy: 84.21%
Accuracy: 85.96%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
Accuracy: 87.72%
